<a href="https://colab.research.google.com/github/AndreKaled/trab3_bd1/blob/main/notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Descrição

# Parte 1
Esta primeira parte do trabalho tem dois objetivos principais. O primeiro é verificar alguns parâmetros do hardware e do software utilizado e o segundo é que os estudantes preparem e se familiarizar com o ambiente do PostgreSQL

## Tarefa 1 - Identificação do sistema
Identifique o sistema que será usado para os experimentos, incluindo informações sobre o hardware e o Sistema Operacional utilizado. Sobre o tipo de processador, quantidade de memória RAM, tamanho do disco. Devem ser também apresentadas informações sobre as caches existentes. Sobre o Sistema Operacional, que devem ser Linux, incluir informações sobre qual a distribuição usada, versão do sistema, versão do Kernel, etc.

**O que entregar:** As informações pedidas devem ser apresentadas no jupyter notebook

## Tarefa 2 - Verificação de parâmetros de armazenamento
a) Verifique no disco que será usado para os experimentos no laboratório os seguintes parâmetros: Nr. de superfícies, cilindros, setores por trilha, velocidade de rotação, latência rotacional; tempos de seek médio, máximo e mínimo; tempo para a próxima trilha; e taxa de transferência.
b) Utilizando o comando “stat” do Linux, verifique os parâmetros dos parâmetros de S.O. que serão utilizados para o disco.
c) Verifique o tamanho de bloco utilizado e mostre como alterar o tamanho dos blocos

**O que entregar:** Os resultados de cada verificação devem ser preenchidas no jupyter notebook

## Tarefa 3 - Geração de um BD para testes
Essa tarefa consiste na criação de banco de dados e povoamento destas tabelas com dados sintéticos. Para a definição do esquema das tabelas e os dados a serem carregados usaremos a especificação e os utilitários fornecidos pelos [Benchmark TPC-H](http://www.tpc.org/tpch/).
Para geração do BD no PostgreSQL, siga as instruções disponíveis em: [https://github.com/foliveirafilho/tpch-pgsql](https://github.com/foliveirafilho/tpch-pgsql)

**O que entregar:** Devem ser apresentadas as saídas da execução dos scripts de geração

## Tarefa 3 - Execução de Consultas
A segunda tarefa deste trabalho consiste em executar e analisar um conjunto de consultas analíticas do benchmark TPC-H sobre o banco de dados gerado na Tarefa 1, observando o comportamento do PostgreSQL em diferentes tipos de operações (filtros, junções, agregações e ordenações). O objetivo é compreender como o otimizador escolhe planos de execução e medir o tempo de resposta de consultas típicas de um sistema de apoio à decisão.
O TPC-H define 22 consultas padronizadas (Q1 a Q22), mas para este trabalho será utilizado um subconjunto representativo composto por 8 consultas que abrangem diferentes padrões de acesso.
Consultas a executar: Q1, Q3, Q5, Q6, Q7, Q9, Q10 e Q12. As consultas originais podem ser obtidos no [repositório oficial do benchmark](https://github.com/memsql/benchmarks-tpc/blob/master/tpch/queries.sql). Essas consultas devem ser executadas no mesmo banco TPC-H criado na Tarefa 1, utilizando o PostgreSQL.
Para cada consulta:
* Execute o comando SQL completo no PostgreSQL.
* Mostre as 10 primeiras linhas do resultado (ou um trecho representativo).
* Execute o comando EXPLAIN ANALYZE antes da consulta para exibir o plano de execução e o tempo real medido O comando EXPLAIN ANALYZE mostra as etapas internas do plano escolhido pelo otimizador, o número de linhas processadas e o tempo de cada operação.
* Registre o tempo total de execução e comente brevemente o tipo de acesso observado (por exemplo, varredura sequencial, uso de índice, junções em hash, etc.).
**O que entregar:** Devem ser apresentados os resultados acima

# Parte 2
O objetivo desta parte do trabalho é analisar o comportamento dos índices das tabelas do SGBD através do exame e análise das tabelas de estatísticas para consultas SQL sobre uma tabela criada com dados aleatórios

## Tarefa 5 - Preparação da Tabela Aleatória
Criar uma tabela com uma chave simples e alguns dados de exemplo. Cada valor de chave é um número incremental e está associado a com valores que variam de 0 até 10:
```sql
DROP TABLE IF EXISTS t;
CREATE TABLE t (k serial PRIMARY KEY, v integer);
INSERT INTO t(v)
SELECT trunc(random() * 10) FROM generate_series(1,100000);
```

**O que entregar:** Imprimir os valores das 10 primeiras tuplas da tabela, ordenando por k.

## Tarefa 6 - Páginas criadas
Verifique quantas páginas com blocos foram criadas para a tabela da Tarefa 5.
Commando: `SELECT relname, relpages, reltuples FROM pg_class WHERE relname='t';`

**O que entregar:** Imprimir o resultado do comando SQL.

## Tarefa 7 – Blocos
Verifique quantos blocos foram efetivamente usados numa consulta
Comando:
```sql
SELECT pg_sleep(1);
\pset x on
SELECT * FROM pg_stats WHERE relname='t';
SELECT pg_stat_reset();
\pset x off
```

Observação: Em algumas versões do PostgreSQL, o atributo é chamado de **tablename** em vez de **relname**.

**O que entregar:** Imprimir o resultado do comando SQL.

## Tarefa 8 – Índice
Crie um índice para o atributo ‘v’ e realize consultas e criação de índice
Qual o tempo gasto para realizar uma consulta para um valor (tendo a tabela 100000 tuplas)?
Qual o tempo gasto para recriar um índice para o atributo ‘v’?
Remova a tabela ‘t’ e crie novamente com 1.0000.000 de tuplas
Qual o tempo gasto para realizar uma consulta para um valor específico?
Qual o tempo gasto para recriar um índice para o atributo ‘v’?

**O que entregar:** Relatório com o resultado das perguntas

## Tarefa 9 - Fill factor
Quando se cria um novo índice, nem toda entrada no bloco do índice é usada. Um espaço livre é deixado, conforme o parâmetro fillfactor.
Crie novos índices usando fillfactor=60,80,90 e 100. Analise o desempenho de suas consultas usando as mesmas condições da Tarefa 12
```sql
ALTER TABLE foo SET ( fillfactor = 50);
VACUUM FULL foo;
```

**O que entregar:** Relatório com o resultado das perguntas

## Tarefa 10 - Utilize índices com ordem DESC
Repita os testes das Tarefas 8 e 9 usando índices descendentes. Avalie e registre o resultado
Comando: `CREATE INDEX i ON t(v DESC NULLS FIRST);`

**O que entregar:** Relatório com o resultado da avaliação e uma análise dos resultados.

# Parte 3
O objetivo desta parte do trabalho é estudar o comportamento dos otimizadores de consulta dos SGBDs através do exame e análise dos planos de execução para consultas SQL sobre tabelas que serão fornecidos. Será bastante utilizado o comando EXPLAIN ANALYZE, que permite visualizar todas as etapas envolvidas no processamento de uma consulta. Usaremos para isso a tabela “[movies](https://drive.google.com/file/d/1W6wovSsVu4B0OIo_tsSBBHi8WRKQqnat/view?usp=drive_link)”.

## Tarefa 11. Preparação e Verificação do Ambiente
a) Execute o script movie.sql em movies para criar as tabelas e índices e carregar os dados necessários às próximas atividades

b) Verifique no catálogo do banco de dados os seguintes metadados sobre os índices associados às tabelas e apresente-os no relatório: Nome do índice, nome da tabela, altura, número máximo de chaves por bloco, número médio de chaves por bloco, número de blocos folha, número de médio de blocos folha por chave, número médio de blocos de dados por chave, número de linhas e número de chaves distintas.

**O que entregar:** Relatório com os resultados da verificação

### a) configurando e criando tabelas e indices

In [11]:
# instalando ferramentas
!sudo apt update
!sudo apt install postgresql postgresql-contrib

!pip install psycopg2 pandas
!service postgresql start
!sudo -u postgres psql -c "ALTER USER postgres PASSWORD 'postgres';"
!wget -O movies.sql "https://drive.google.com/uc?id=1W6wovSsVu4B0OIo_tsSBBHi8WRKQqnat&export=download"

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 https://cli.github.com/packages stable InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:11 http://security.ubuntu.com/ubuntu jammy-security InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
50 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: Skipping acquire of configured file 'main/source/Sources' as re

In [31]:
import psycopg2
from psycopg2 import errors

# conecta no banco postgres
conn = psycopg2.connect(
    dbname="postgres",
    user="postgres",
    password="postgres",
    host="localhost"
)
conn.autocommit = True
cur = conn.cursor()

# criar usuário e database
try:
    cur.execute("CREATE USER icomp WITH PASSWORD 'icomp123';")
except:
    print("Usuário icomp já existe.")

try:
    cur.execute("CREATE DATABASE icomp OWNER icomp;")
except:
    print("Banco icomp já existe.")

# agora conecta no banco para criar extensões
conn.close()
conn = psycopg2.connect(
    dbname="icomp",
    user="postgres",
    password="postgres",
    host="localhost"
)
conn.autocommit = True
cur = conn.cursor()

# extensões e permissões
cur.execute("CREATE EXTENSION IF NOT EXISTS pageinspect;")
cur.execute("CREATE EXTENSION IF NOT EXISTS pgstattuple;")

cur.execute("GRANT ALL PRIVILEGES ON DATABASE icomp TO icomp;")
cur.execute("GRANT ALL ON SCHEMA public TO icomp;")
cur.execute("ALTER ROLE icomp SET search_path = public;")

# printa extensoes ativas
#cur.execute("SELECT extname FROM pg_extension;")
#print(cur.fetchall())
cur.close()
conn.close()

conn = psycopg2.connect(
    dbname="icomp",
    user="icomp",
    password="icomp123",
    host="localhost"
)
cur = conn.cursor()

# carrega o script, usa senha do usuario icomp
!psql -h localhost -U icomp -d icomp -f movie.sql

#with open("movie.sql", "r") as f:
#    sql_script = f.read()

#cur.execute(sql_script)
#conn.commit()


Usuário icomp já existe.
Banco icomp já existe.
Password for user icomp: 
psql:movie.sql:1: ERROR:  table "movie" does not exist
CREATE TABLE
CREATE INDEX
CREATE INDEX
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 

### b)

In [32]:
sql = """
SELECT
    i.relname AS "Nome do Índice",
    t.relname AS "Tabela",
    meta.level + 1 AS "Altura",

    -- 2. MÁX CHAVES/BLOCO (Calculado com o Tamanho Exato do Tipo de Dado)
    FLOOR(current_setting('block_size')::int /
        CASE
            -- SMALLINT (2B) + Overhead (12B) = 14B
            WHEN i.relname = 'movie_key' THEN 14.0
            -- INTEGER (4B) + Overhead (12B) = 16B
            WHEN i.relname = 'movie_votes' THEN 16.0
            -- VARCHAR (usamos 32.0 como uma estimativa segura para o overhead de texto)
            ELSE 32.0
        END
    ) AS "Máx Chaves/Bloco",

    -- 3. MÉDIO CHAVES/BLOCO (Total Linhas / Total Páginas Folha do Disco)
    ROUND(c.reltuples / GREATEST(stat.leaf_pages, 1)) AS "Médio Chaves/Bloco",

    -- 4. BLOCOS FOLHA
    stat.leaf_pages AS "Blocos Folha",

    -- 5. MÉDIO BLOCOS FOLHA/CHAVE (Custo de descida = Altura)
    (meta.level + 1) AS "Médio Blocos Folha/Chave",

    -- 6. MÉDIO BLOCOS DADOS/CHAVE
    1 AS "Médio Blocos Dados/Chave",

    -- 7. NÚMERO DE LINHAS
    c.reltuples AS "Número de Linhas",

    -- 8. CHAVES DISTINTAS
    CASE
        WHEN i.relname = 'movie_key' THEN (SELECT COUNT(DISTINCT id) FROM movie)
        WHEN i.relname = 'movie_title' THEN (SELECT COUNT(DISTINCT title) FROM movie)
        WHEN i.relname = 'movie_votes' THEN (SELECT COUNT(DISTINCT votes) FROM movie)
    END AS "Chaves Distintas"

FROM pg_class t
JOIN pg_index ix ON t.oid = ix.indrelid
JOIN pg_class i ON i.oid = ix.indexrelid
JOIN pg_class c ON c.oid = i.oid
CROSS JOIN LATERAL bt_metap(CAST(i.relname AS text)) meta
CROSS JOIN LATERAL pgstatindex(CAST(i.relname AS text)) stat
WHERE t.relname = 'movie';
"""

cur.execute(sql)
rows = cur.fetchall()
cols = [desc[0] for desc in cur.description]

df = pd.DataFrame(rows, columns=cols)

print(df)

InsufficientPrivilege: permission denied for function pgstatindex


## Tarefa 12. Consultas por intervalo e índices secundários
a) Escreva uma consulta em SQL sobre o atributo VOTES da tabela MOVIE que recupera um número pequeno de tuplas (<10 tuplas); Execute o comando EXPLAIN ANALYZE sobre esta consulta e apresente os resultados.

b) Escreva uma consulta em SQL sobre o atributo VOTES da tabela MOVIE que recupera um número grande de tuplas (>80% das tuplas). Execute o comando EXPLAIN ANALYZE sobre esta consulta e apresente os resultados.

c) Explique porque o índice sobre VOTES não é sempre usado nas consultas sobre este atributo.

**O que entregar:** Relatório com as respostas das questões.

In [33]:
def run_explain(query):
    conn = psycopg2.connect(
        dbname="icomp",
        user="postgres",
        password="postgres",
        host="localhost"
    )
    cur = conn.cursor()
    cur.execute(f"EXPLAIN ANALYZE {query}")
    result = cur.fetchall()
    conn.close()

    # psql mostra linha por linha
    for row in result:
        print(row[0])

run_explain("SELECT * FROM movie WHERE votes BETWEEN 5000 AND 5005;")
run_explain("SELECT * FROM movie WHERE votes >= 100;")


Index Scan using movie_votes on movie  (cost=0.28..8.32 rows=2 width=30) (actual time=0.014..0.014 rows=0 loops=1)
  Index Cond: ((votes >= 5000) AND (votes <= 5005))
Planning Time: 0.829 ms
Execution Time: 0.097 ms
Seq Scan on movie  (cost=0.00..38.05 rows=1844 width=30) (actual time=0.009..0.503 rows=1844 loops=1)
  Filter: (votes >= 100)
Planning Time: 0.774 ms
Execution Time: 0.666 ms


### a)
ANALYZE de `SELECT * FROM movie WHERE votes BETWEEN 5000 AND 5005;`:
```bash
                                                     QUERY PLAN                                                     
--------------------------------------------------------------------------------------------------------------------
 Index Scan using movie_votes on movie  (cost=0.28..8.32 rows=2 width=30) (actual time=0.038..0.038 rows=0 loops=1)
   Index Cond: ((votes >= 5000) AND (votes <= 5005))
 Planning Time: 0.439 ms
 Execution Time: 0.083 ms
(4 rows)
```

### b)
ANALYZE de `SELECT * FROM movie WHERE votes >= 100`:
```bash
                                              QUERY PLAN                                               
-------------------------------------------------------------------------------------------------------
 Seq Scan on movie  (cost=0.00..38.05 rows=1844 width=30) (actual time=0.011..0.514 rows=1844 loops=1)
   Filter: (votes >= 100)
 Planning Time: 0.649 ms
 Execution Time: 0.648 ms
(4 rows)
```

### c)
Um índice é útil quando ele filtra poucas linhas. Se na consulta queremos quase tudo, o índice vira "peso extra" e o banco prefere busca sequencial do que buscar por index + sequencial.

## Tarefa 13. Comparações de operadores de agregação.
Considere as seguintes consultas em SQL, sobre o atributo VOTES, as quais são equivalentes:
```sql
* SELECT title FROM movie WHERE votes >= (SELECT MAX(votes) FROM movie);
* SELECT title FROM movie WHERE votes >= ALL (SELECT votes FROM movie) ;
```

a) Apresente o resultado do comando EXPLAIN ANALYZE sobre as duas consultas acima

b) Existe alguma diferença entre os planos de consultas? Qual das duas é mais eficiente? Explique

**O que entregar:** Relatório com as respostas das questões

In [34]:
run_explain("SELECT title FROM movie WHERE votes >= (SELECT MAX(votes) FROM movie);")
run_explain("SELECT title FROM movie WHERE votes >= ALL (SELECT votes FROM movie);")

Index Scan using movie_votes on movie  (cost=0.61..35.37 rows=615 width=16) (actual time=0.052..0.053 rows=1 loops=1)
  Index Cond: (votes >= $1)
  InitPlan 2 (returns $1)
    ->  Result  (cost=0.32..0.33 rows=1 width=4) (actual time=0.042..0.043 rows=1 loops=1)
          InitPlan 1 (returns $0)
            ->  Limit  (cost=0.28..0.32 rows=1 width=4) (actual time=0.041..0.041 rows=1 loops=1)
                  ->  Index Only Scan Backward using movie_votes on movie movie_1  (cost=0.28..76.55 rows=1844 width=4) (actual time=0.040..0.040 rows=1 loops=1)
                        Index Cond: (votes IS NOT NULL)
                        Heap Fetches: 0
Planning Time: 0.707 ms
Execution Time: 0.142 ms
Seq Scan on movie  (cost=0.00..43620.99 rows=922 width=16) (actual time=0.800..1.249 rows=1 loops=1)
  Filter: (SubPlan 1)
  Rows Removed by Filter: 1843
  SubPlan 1
    ->  Materialize  (cost=0.00..42.66 rows=1844 width=4) (actual time=0.000..0.000 rows=2 loops=1844)
          ->  Seq Scan on mov

### a)
ANALYZE de `SELECT title FROM movie WHERE votes >= (SELECT MAX(votes) FROM movie);`:
```bash
                                                                            QUERY PLAN                                                                             
-------------------------------------------------------------------------------------------------------------------------------------------------------------------
 Index Scan using movie_votes on movie  (cost=0.62..35.38 rows=615 width=16) (actual time=0.075..0.077 rows=1 loops=1)
   Index Cond: (votes >= $1)
   InitPlan 2 (returns $1)
     ->  Result  (cost=0.33..0.34 rows=1 width=4) (actual time=0.063..0.064 rows=1 loops=1)
           InitPlan 1 (returns $0)
             ->  Limit  (cost=0.28..0.33 rows=1 width=4) (actual time=0.060..0.061 rows=1 loops=1)
                   ->  Index Only Scan Backward using movie_votes on movie movie_1  (cost=0.28..94.55 rows=1844 width=4) (actual time=0.058..0.059 rows=1 loops=1)
                         Index Cond: (votes IS NOT NULL)
                         Heap Fetches: 1
 Planning Time: 0.761 ms
 Execution Time: 0.166 ms
(11 rows)

```

ANALYZE de `SELECT title FROM movie WHERE votes >= ALL (SELECT votes FROM movie);`:
```bash
                                                         QUERY PLAN                                                         
----------------------------------------------------------------------------------------------------------------------------
 Seq Scan on movie  (cost=0.00..43620.99 rows=922 width=16) (actual time=1.018..1.640 rows=1 loops=1)
   Filter: (SubPlan 1)
   Rows Removed by Filter: 1843
   SubPlan 1
     ->  Materialize  (cost=0.00..42.66 rows=1844 width=4) (actual time=0.000..0.001 rows=2 loops=1844)
           ->  Seq Scan on movie movie_1  (cost=0.00..33.44 rows=1844 width=4) (actual time=0.004..0.398 rows=1844 loops=1)
 Planning Time: 0.650 ms
 Execution Time: 1.718 ms
(8 rows)

```


### b)
Sim, existem diferenças:

Query com MAX(votes) - Mais eficiente

* Usa o índice movie_votes.
* Como o índice é ordenado, o PostgreSQL faz um Index Only Scan Backward: ele lê diretamente a última entrada do índice para descobrir o maior valor.
* Isso evita percorrer a tabela inteira; o custo é muito baixo e praticamente constante.
* Depois disso, filtra usando esse valor máximo direto no índice.

Query com ALL (…) - Menos eficiente

* O PostgreSQL não otimiza automaticamente para “máximo”.
* Ele precisa comparar cada linha com o conjunto completo, depende do tamanho da tabela existente.
* Esse processo depende linearmente do tamanho da tabela, sem conseguir usar o índice.

## Tarefa 14. Consultas com Junção e Seleção
Considere as duas consultas equivalentes em SQL a seguir, as quais retornam os filmes com mais votos que “Star Wars”
```sql
* SELECT title FROM movie WHERE votes > (SELECT votes FROM movie WHERE title = 'Star Wars');
* SELECT m1.title FROM movie m1, movie m2 WHERE m1.votes > m2.votes AND m2.title = 'Star Wars';
```

a) Apresente o resultado do comando EXPLAIN ANALYZE sobre as duas consultas acima

b) Existe alguma diferença entre os planos de consultas? Qual das duas é mais eficiente? Explique.

**O que entregar:** Relatório com as respostas das questões.

In [35]:
run_explain("SELECT title FROM movie WHERE votes > (SELECT votes FROM movie WHERE title = 'Star Wars');")
run_explain("SELECT m1.title FROM movie m1, movie m2 WHERE m1.votes > m2.votes AND m2.title = 'Star Wars';")

Index Scan using movie_votes on movie  (cost=8.57..43.34 rows=615 width=16) (actual time=0.045..0.045 rows=0 loops=1)
  Index Cond: (votes > $0)
  InitPlan 1 (returns $0)
    ->  Index Scan using movie_title on movie movie_1  (cost=0.28..8.29 rows=1 width=4) (actual time=0.028..0.028 rows=1 loops=1)
          Index Cond: ((title)::text = 'Star Wars'::text)
Planning Time: 0.536 ms
Execution Time: 0.093 ms
Nested Loop  (cost=0.56..49.49 rows=615 width=16) (actual time=0.034..0.035 rows=0 loops=1)
  ->  Index Scan using movie_title on movie m2  (cost=0.28..8.29 rows=1 width=4) (actual time=0.021..0.022 rows=1 loops=1)
        Index Cond: ((title)::text = 'Star Wars'::text)
  ->  Index Scan using movie_votes on movie m1  (cost=0.28..35.04 rows=615 width=20) (actual time=0.010..0.010 rows=0 loops=1)
        Index Cond: (votes > m2.votes)
Planning Time: 0.694 ms
Execution Time: 0.091 ms


### a)
ANALYZE de `SELECT title FROM movie WHERE votes > (SELECT votes FROM movie WHERE title = 'Star Wars');`:
```bash
                                                            QUERY PLAN                                                             
-----------------------------------------------------------------------------------------------------------------------------------
 Index Scan using movie_votes on movie  (cost=8.57..43.34 rows=615 width=16) (actual time=0.055..0.056 rows=0 loops=1)
   Index Cond: (votes > $0)
   InitPlan 1 (returns $0)
     ->  Index Scan using movie_title on movie movie_1  (cost=0.28..8.29 rows=1 width=4) (actual time=0.039..0.040 rows=1 loops=1)
           Index Cond: ((title)::text = 'Star Wars'::text)
 Planning Time: 0.675 ms
 Execution Time: 0.122 ms
(7 rows)
```

ANALYZE de `SELECT m1.title FROM movie m1, movie m2 WHERE m1.votes > m2.votes AND m2.title = 'Star Wars';`:
```bash
                                                           QUERY PLAN                                                           
--------------------------------------------------------------------------------------------------------------------------------
 Nested Loop  (cost=0.56..49.49 rows=615 width=16) (actual time=0.036..0.036 rows=0 loops=1)
   ->  Index Scan using movie_title on movie m2  (cost=0.28..8.29 rows=1 width=4) (actual time=0.025..0.025 rows=1 loops=1)
         Index Cond: ((title)::text = 'Star Wars'::text)
   ->  Index Scan using movie_votes on movie m1  (cost=0.28..35.04 rows=615 width=20) (actual time=0.008..0.008 rows=0 loops=1)
         Index Cond: (votes > m2.votes)
 Planning Time: 0.564 ms
 Execution Time: 0.085 ms
(7 rows)

```

### b)
Consulta 1 (Subquery): O otimizador utilizou um InitPlan. Ele percebeu que a subconsulta (SELECT votes FROM movie WHERE title = 'Star Wars') é independente da consulta externa. Portanto, ele executou essa subconsulta apenas uma vez, armazenou o resultado como um parâmetro (representado por \$0), e depois realizou um Index Scan simples na tabela movie buscando valores maiores que \$0.

Consulta 2 (Join): O otimizador utilizou um Nested Loop (for dentro de outro for). Ele primeiro buscou a linha referente a 'Star Wars' (tabela m2) usando um índice e para essa linha encontrada, iterou sobre a tabela m1 usando o índice de votos para encontrar os correspondentes.

As duas consultas acabam fazendo praticamente o mesmo trabalho, mas a primeira resolve a parte variável antes e transforma a condição em algo constante. Isso elimina o Nested Loop e produz um plano mais simples.
Mesmo que, nesse caso, o tempo de execução seja parecida, a consulta 1 é a **mais eficiente** por evitar loops desnecessários.



## Tarefa 15. Casamento de Strings e Índices
Considere as seguintes consultas SQL sobre o atributo TITLE usando o operador LIKE.
```sql
* SELECT title FROM movie WHERE title LIKE 'I%';
* SELECT title FROM movie WHERE substr(title, 1, 1) = 'I';
* SELECT title FROM movie WHERE title LIKE '%A';
```
a) Apresente o resultado do comando explain sobre as três consultas acima

b) Qual das três apresenta o menor custo? Porque?

c) O índice sobre TITLE foi usado para todas elas? Justifique.

**O que entregar:** Relatório com as respostas das questões.

In [36]:
run_explain("SELECT title FROM movie WHERE title LIKE 'I%';")
run_explain("SELECT title FROM movie WHERE substr(title, 1, 1) = 'I';")
run_explain("SELECT title FROM movie WHERE title LIKE '%A';")

Seq Scan on movie  (cost=0.00..38.05 rows=18 width=16) (actual time=0.011..0.279 rows=25 loops=1)
  Filter: ((title)::text ~~ 'I%'::text)
  Rows Removed by Filter: 1819
Planning Time: 0.570 ms
Execution Time: 0.311 ms
Seq Scan on movie  (cost=0.00..42.66 rows=9 width=16) (actual time=0.015..0.420 rows=25 loops=1)
  Filter: (substr((title)::text, 1, 1) = 'I'::text)
  Rows Removed by Filter: 1819
Planning Time: 0.463 ms
Execution Time: 0.456 ms
Seq Scan on movie  (cost=0.00..38.05 rows=18 width=16) (actual time=0.014..0.329 rows=30 loops=1)
  Filter: ((title)::text ~~ '%A'::text)
  Rows Removed by Filter: 1814
Planning Time: 0.484 ms
Execution Time: 0.361 ms


### a)
Consulta 1 (LIKE 'I%'):
```bash
                                            QUERY PLAN                                             
---------------------------------------------------------------------------------------------------
 Seq Scan on movie  (cost=0.00..38.05 rows=18 width=16) (actual time=0.009..0.184 rows=25 loops=1)
   Filter: ((title)::text ~~ 'I%'::text)
   Rows Removed by Filter: 1819
 Planning Time: 0.457 ms
 Execution Time: 0.221 ms
(5 rows)
```

Consulta 2 (substr(title, 1, 1) = 'I'):
```bash
                                            QUERY PLAN                                            
--------------------------------------------------------------------------------------------------
 Seq Scan on movie  (cost=0.00..42.66 rows=9 width=16) (actual time=0.013..0.306 rows=25 loops=1)
   Filter: (substr((title)::text, 1, 1) = 'I'::text)
   Rows Removed by Filter: 1819
 Planning Time: 0.436 ms
 Execution Time: 0.341 ms
(5 rows)
```

Consulta 3 (LIKE '%A'):
```bash
                                           QUERY PLAN                                             
---------------------------------------------------------------------------------------------------
 Seq Scan on movie  (cost=0.00..38.05 rows=18 width=16) (actual time=0.017..0.385 rows=30 loops=1)
   Filter: ((title)::text ~~ '%A'::text)
   Rows Removed by Filter: 1814
 Planning Time: 0.644 ms
 Execution Time: 0.426 ms
(5 rows)
```

### b)
Todas as consultas fizeram Seq Scan, mas as consultas 1 e 3 tiveram o custo menor em relação à consulta 2 pois não executa uma chamada de função `substr()` para cada uma das 1844 linhas, aumentando o custo estimado em comparação com o operador `LIKE` das outras consultas.

### c)
Não, todas usaram Seq Scan (banco leu linha a linha)

Consulta `LIKE 'I%'`:
* Pode ser usado a B-tree para busca por estar ordenada alfabéticamente, mas o Postgres decidiu que a tabela tem poucas tuplas e um Seq Scan é mais barato.

Consulta `substr(title, 1, 1) = 'I'`:
* O indice guarda o valor de `title` e não do resultado da função `substr()`, como a consulta aplica a função o banco de dados não consegue olhar pro indice e achar a resposta, sendo obrigado a calcular a função `substr()` em todas as tuplas para cumprir com a clausula WHERE.

Consulta `LIKE '%A'`:
* A B-tree indexada é ordenada em ordem alfabética, mas as palavras terminadas em 'A' estão por toda parte da àrvore, obrigando a leitura sequencial.

## Tarefa 16. Verificação da hipótese de distribuição uniforme na estimativa de seletividade
Considere as seguintes consultas sobre o atributo TITLE da tabela MOVIE
* SELECT title FROM movie WHERE votes < 1000;
* SELECT title FROM movie WHERE votes > 40000
a) Apresente o resultado do comando explain sobre as duas consultas acima. Explique o resultado.
b) Compare o número de tuplas selecionadas por cada consulta. Qual das duas tem a menor seletividade?

**O que entregar:** Relatório com as respostas das questões.

# Parte 4
O objetivo desta parte do trabalho é experimentar estratégias para utilização de transações e níveis de
isolamento em SGBDs relacionais. As tarefas envolvem uma simulação de um sistema de reservas de
passagem áreas.
Considere a seguinte tabela que registra os assentos reservados em um vôo:
Assentos(num_voo,disp)
onde num_voo um número inteiro de 1 a 200 e disp é um atributo booleano cujo valor é true se o
assento estiver vago e false caso contrário. O valor inicial é true.
A reserva de um assento é feita em três passos:
Passo 1. O sistema recupera a lista dos assentos disponíveis.
Passo 2. O cliente escolhe o assento. Esse passo deve ser simulado pela escolha aleatória de um
dos assentos disponíveis, levando para isso um tempo de escolha de 1 segundo.
Passo 3. O sistema registra a reserva do assento escolhido, atualizando o valor de disp para
false.
Cada assento é reservado individualmente. Duas versões diferentes do processo de reserva devem ser
implementadas.
Versão a. A reserva é implementada como uma única transação que inclui os três passos acima.
Versão b. A reserva inclui uma transação para o Passo 1 e outra para o Passo 3. O Passo 2 não
faz parte das transações, mas deve ser executado.
Agentes de viagens são responsáveis por realizar as reservas de 200 clientes no total. A atividade de um
agente de viagens é simulada por uma thread.
Experimentos devem ser realizados simulando a atuação de k agentes de viagem trabalhando
simultaneamente, onde k = 1,2,4,6,8 e 10. Cada agente/thread faz uma reserva de cada vez. As threads
devem ser reiniciadas até que todos os 200 clientes tenham seus assentos reservados.
Dois conjuntos de experimentos devem ser feitos usando dois níveis de isolamento: “read committed” e
“serializable”. Nos dois casos, o sistema deve ser configurado para realizar bloqueios a nível de tupla
(linha).
As implementações devem ser feitas em Python3.8+ usando o SGBD PostgreSQL.
Considerando o descrito acima, execute as seguintes tarefas:

## Tarefa 17. Implemente as versões a e b do processo de reserva.

**O que entregar:** Código fonte Python3.8+ documentado das duas versões.

## Tarefa 18. Apresente gráficos de linha onde, para cada valor de k (número de agentes) no eixo x, temos no eixo y o tempo necessário para que todos os clientes efetuem suas reservas. Um gráfico diferente deve ser apresentado para cada par de versões da reserva e nível de isolamento.

## Tarefa 19. Apresente uma tabela com o número máximo, mínimo e médio de vezes que um cliente teve que tentar reservar um assento até conseguir, ou seja, o número de vezes que uma reserva teve que ser refeita. A tabela considera as variações de k, versão de reserva e nível de isolamento.

## Tarefa 20. Apresente uma análise dos resultados obtidos em cada versão de reserva e tipo de isolamento, explicando as diferenças entre resultados.